In [1]:
import pandas as pd
import sqlite3
from sqlalchemy import create_engine
import random
import datetime

In [2]:
pd.options.display.max_colwidth = 50

In [3]:
engine = create_engine("mysql://root@localhost/parkzen")

In [4]:
with engine.connect() as conn, conn.begin():
    users = pd.read_sql_table(table_name='auth_user', con=conn)

In [5]:
users

,id,password,last_login,is_superuser,username,first_name,last_name,email,is_staff,is_active,date_joined
0,1,pbkdf2_sha256$100000$mMR4QmyIZ7A4$i2a8/l0ZvzPA...,2018-05-05 08:39:27.772542,1,navaneethan,,,,1,1,2018-05-05 08:39:07.685361
1,2,pbkdf2_sha256$100000$Ej8iMPPaqMuP$fQnz1tnOD5Lq...,NaT,0,ankit,,,,0,1,2018-05-05 08:39:43.404826
2,3,pbkdf2_sha256$100000$OPPxp8HYFZbG$blxWz460JopK...,NaT,0,hruthika,,,,0,1,2018-05-05 08:39:56.681942


In [6]:
with engine.connect() as conn, conn.begin():
    lat_lon = pd.read_sql_table(table_name='app_location', con=conn)

In [7]:
lat_lon.shape

(50, 5)

In [9]:
lat_lon['address'].head()

0      56 & 57, Mysore Rd, Jayanagar, Bangalore 560121
1    56 & 57, Mysore Rd, Indiranagar, Bangalore 560078
2      B32, Sampige Rd, Malleshwaram, Bangalore 560103
3    56 & 57, Sampige Rd, Malleshwaram, Bangalore 5...
4           123, Mysore Rd, JP Nagar, Bangalore 560078
Name: address, dtype: object

In [10]:
lat_lon.head()

,id,lat,lon,user_id,address
0,101,12.907909,77.607073,2,"56 & 57, Mysore Rd, Jayanagar, Bangalore 560121"
1,102,12.907506,77.607736,3,"56 & 57, Mysore Rd, Indiranagar, Bangalore 560078"
2,103,12.912361,77.602853,1,"B32, Sampige Rd, Malleshwaram, Bangalore 560103"
3,104,12.906575,77.602821,3,"56 & 57, Sampige Rd, Malleshwaram, Bangalore 5..."
4,105,12.911095,77.599719,1,"123, Mysore Rd, JP Nagar, Bangalore 560078"


In [11]:
lat_lon_users = pd.merge(left=lat_lon, right=users, left_on='user_id', right_on='id')

In [12]:
lat_lon_users.shape

(50, 16)

In [13]:
lat_lon_users.head()

,id_x,lat,lon,user_id,address,id_y,password,last_login,is_superuser,username,first_name,last_name,email,is_staff,is_active,date_joined
0,101,12.907909,77.607073,2,"56 & 57, Mysore Rd, Jayanagar, Bangalore 560121",2,pbkdf2_sha256$100000$Ej8iMPPaqMuP$fQnz1tnOD5Lq...,NaT,0,ankit,,,,0,1,2018-05-05 08:39:43.404826
1,107,12.905191,77.608909,2,"B32, Mysore Rd, Jayanagar, Bangalore 560002",2,pbkdf2_sha256$100000$Ej8iMPPaqMuP$fQnz1tnOD5Lq...,NaT,0,ankit,,,,0,1,2018-05-05 08:39:43.404826
2,115,12.907084,77.608015,2,"123, MG Road, JP Nagar, Bangalore 560002",2,pbkdf2_sha256$100000$Ej8iMPPaqMuP$fQnz1tnOD5Lq...,NaT,0,ankit,,,,0,1,2018-05-05 08:39:43.404826
3,117,12.907424,77.605659,2,"49A, MG Road, JP Nagar, Bangalore 560121",2,pbkdf2_sha256$100000$Ej8iMPPaqMuP$fQnz1tnOD5Lq...,NaT,0,ankit,,,,0,1,2018-05-05 08:39:43.404826
4,118,12.908196,77.604029,2,"#77, Bannerghatta Road, Indiranagar, Bangalore...",2,pbkdf2_sha256$100000$Ej8iMPPaqMuP$fQnz1tnOD5Lq...,NaT,0,ankit,,,,0,1,2018-05-05 08:39:43.404826


In [14]:
a = {}

In [15]:
num_slots = 3
random.sample(population=range(9, 17), k=num_slots)

[16, 12, 13]

In [18]:
slots = []
year = 2018
month = 5
minute = 0
second = 0
for i in range(lat_lon_users.shape[0]):
    rate = random.randrange(start=50, stop=1000, step=50)
    
    start_times = {}
    end_times = {}
    day = random.randrange(start=6, stop=9)
    start_hour = random.randrange(start=9, stop=17)
    end_offset = random.randrange(start=1, stop=5)
    end_hour = start_hour+end_offset

    start_time = datetime.datetime(year, month, day, start_hour, minute, second)
    end_time = datetime.datetime(year, month, day, end_hour, minute, second)

    slots.append(
        (
            lat_lon_users.loc[i, 'id_x'],
            lat_lon_users.loc[i, 'user_id'],
            start_time,
            end_time,
            rate
        )
    )

In [19]:
one_slot_per_spot = slots

In [20]:
len(one_slot_per_spot)

50

In [21]:
one_slot_per_spot_df = pd.DataFrame(one_slot_per_spot, columns=['location_id', 'user_id', 'start', 'end', 'rate'])

In [22]:
one_slot_per_spot_df.sample(n=5)

,location_id,user_id,start,end,rate
28,127,3,2018-05-07 09:00:00,2018-05-07 12:00:00,950
40,124,1,2018-05-07 10:00:00,2018-05-07 11:00:00,500
37,112,1,2018-05-07 15:00:00,2018-05-07 16:00:00,600
8,129,2,2018-05-06 12:00:00,2018-05-06 16:00:00,400
24,120,3,2018-05-08 10:00:00,2018-05-08 11:00:00,850


## Insert into DB

In [23]:
with engine.connect() as conn, conn.begin():
    one_slot_per_spot_df.to_sql(name='app_timeslot', con=conn, if_exists='append', index=False)

# Scratch

## Get lat-long

In [ ]:
def get_lat_lon_data(fname='../random_lat_long.csv'):
    lat_long = pd.read_csv(fname)
    lat_long.rename(columns={'id': 'user_id', 'long': 'lon'}, inplace=True)
    lat_long.drop(labels='random', inplace=True, axis=1)
    lat_long['address'] = ['123 Bannerghatta Road, JP Nagar, Bangalore 560076 Karnataka'] * lat_long.shape[0]

    usernames = ['navaneethan', 'ankit', 'hruthika']*17
    usernames = usernames[:-1]
    random.shuffle(usernames)

    username_map = {'navaneethan': 1, 'ankit': 2, 'hruthika': 3}

    lat_long['user_id'] = usernames

    lat_long['user_id'] = lat_long['user_id'].apply(lambda u: username_map[u])
    
    return lat_long

In [ ]:
lat_long.head()

## This is the old way of doing things - `pandas` does not support it

`import MySQLdb`

`conn = MySQLdb.connect(host='localhost', user='root', password='')
cursor = conn.cursor()
cursor.execute('show databases')
for database in cursor:
    print(database)
cursor.execute('use parkzen')
cursor.execute('desc app_location')
for location in cursor:
    print(location)`